In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client_openai = OpenAI()

In [2]:
chat_completion = client_openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Hello world!"},
    ],
)
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [3]:
def openai_chat_completion(messages):
    return client_openai.chat.completions.create(
        model="gpt-4o", temperature=0.0, messages=messages
    )

In [4]:
class Agent:
    def __init__(self, llm_chat_completion, system=""):
        self.system = system
        self.messages = []
        self.llm_chat_completion = llm_chat_completion
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.llm_chat_completion(self.messages)
        return completion.choices[0].message.content

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [6]:
def calculate(what):
    return eval(what)


def average_dog_weight(breed):
    if breed in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif breed in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif breed in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

In [7]:
not_so_automted_bot = Agent(openai_chat_completion, prompt)

In [8]:
result = not_so_automted_bot("How much does a Toy Poodle weigh?")
result

'Thought: I should look up the average weight of a Toy Poodle using the available action.\nAction: average_dog_weight: Toy Poodle\nPAUSE'

In [9]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [10]:
next_prompt = f"Observation: {result}"

In [11]:
not_so_automted_bot(next_prompt)

'Answer: A Toy Poodle weighs, on average, 7 lbs.'

In [12]:
not_so_automted_bot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [13]:
not_so_automted_bot = Agent(openai_chat_completion, prompt)

In [14]:
not_so_automted_bot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'}]

In [15]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
not_so_automted_bot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then sum these weights to get the combined weight of the two dogs.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [16]:
not_so_automted_bot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user',
  'content': 'I have 2 dogs,

In [17]:
next_prompt = f"Observation: {average_dog_weight("Border Collie")}"
next_prompt

'Observation: a Border Collies average weight is 37 lbs'

In [18]:
not_so_automted_bot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [19]:
next_prompt = f"Observation: {average_dog_weight("Scottish Terrier")}"
next_prompt

'Observation: Scottish Terriers average 20 lbs'

In [20]:
not_so_automted_bot(next_prompt)

'Thought: I now have the average weights of both dogs. I need to sum these weights to find their combined weight.\nAction: calculate: 37 + 20\nPAUSE'

In [21]:
next_prompt = f"calculate: {calculate("50 + 20")}"
next_prompt

'calculate: 70'

In [22]:
not_so_automted_bot(next_prompt)

'Observation: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.\n\nAnswer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

In [23]:
import re

action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
/var/folders/1m/pswlhn1n1gv1c0lymv7l947r0000gn/T/ipykernel_8874/1748314687.py:4: SyntaxWarning: invalid escape sequence '\w'
  "^Action: (\w+): (.*)$"


In [24]:
sample_string = """Action: average_dog_weight: Border Collie"""

# Use the pattern to match the sample string
match = action_re.match(sample_string)

# Check if there was a match
if match:
    # Extract the captured groups
    action_type = match.group(1)
    action_details = match.group(2)

    # Print the captured groups
    print(f"Action Type: {action_type}")
    print(f"Action Details: {action_details}")
else:
    print("No match found.")

Action Type: average_dog_weight
Action Details: Border Collie


In [25]:
sample = """Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE"""
for a in sample.split("\n"):
    print(f"- {a}")

- Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
- Action: average_dog_weight: Border Collie
- PAUSE


In [26]:
def query(question, max_turns=5):
    i = 0
    now_automated_bot = Agent(openai_chat_completion, prompt)
    next_prompt = question
    while i < max_turns:
        result = now_automated_bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action in known_actions:
                print(f"-- running: {action}('{action_input}')")
                observation = known_actions[action](action_input)
                print(f"-- observation: {observation}")
                next_prompt = f"Observation: {observation}"
            else:
                raise Exception("Unknown action: {}: {}".format(action, action_input))

        else:
            return

In [27]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
-- running: average_dog_weight('Border Collie')
-- observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
-- running: average_dog_weight('Scottish Terrier')
-- observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights of both dogs. I need to add these weights together to find their combined weight.
Action: calculate: 37 + 20
PAUSE
-- running: calculate('37 + 20')
-- observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


### Anthropic - Claude Sonnet 3.5

In [28]:
import anthropic

client_anthropic = anthropic.Anthropic()

In [29]:
client_anthropic.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="Say hi",
    messages=[{"role": "user", "content": "Hello Claude!"}],
)

Message(id='msg_016mKHzUbgzoAntu1ZwY7uMS', content=[TextBlock(text="Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with?", type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=12, output_tokens=27))

In [30]:
def anthropic_chat_completion(system, messages):
    return client_anthropic.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0.0,
        system=system,
        messages=messages,
    )

In [45]:
class AnthropicAgent:
    def __init__(self, llm_chat_completion, system=""):
        self.system = system
        self.messages = []
        self.llm_chat_completion = llm_chat_completion

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result[0].text

    def execute(self):
        completion = self.llm_chat_completion(self.system, self.messages)
        return completion.content

In [47]:
def anthropic_query(question, max_turns=5):
    i = 0
    bot = AnthropicAgent(anthropic_chat_completion, prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return bot.messages

In [48]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
anthropic_query(question)

Thought: To answer this question, I need to find out the average weight of both a Border Collie and a Scottish Terrier, then add them together. Let's start with the Border Collie.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Great, now I know the average weight of a Border Collie. Next, I need to find out the average weight of a Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now I have the average weights for both dogs. To get their combined weight, I need to add these two numbers together.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Thought: The calculation is complete. I now have all the information needed to answer the question.

Answer: The combined weight of your Border Collie and Scottish Terrier is 57 lbs. Th

[{'role': 'user',
  'content': 'I have 2 dogs, a border collie and a scottish terrier. What is their combined weight'},
 {'role': 'assistant',
  'content': [TextBlock(text="Thought: To answer this question, I need to find out the average weight of both a Border Collie and a Scottish Terrier, then add them together. Let's start with the Border Collie.\n\nAction: average_dog_weight: Border Collie\nPAUSE", type='text')]},
 {'role': 'user',
  'content': 'Observation: a Border Collies average weight is 37 lbs'},
 {'role': 'assistant',
  'content': [TextBlock(text='Thought: Great, now I know the average weight of a Border Collie. Next, I need to find out the average weight of a Scottish Terrier.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE', type='text')]},
 {'role': 'user', 'content': 'Observation: Scottish Terriers average 20 lbs'},
 {'role': 'assistant',
  'content': [TextBlock(text='Thought: Now I have the average weights for both dogs. To get their combined weight, I need to a